In [1]:
####Imports and Setup
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import patsy
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
###########################
import numpy as np
import scipy 
from scipy.interpolate import interp1d
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler
# We are using the BernoulliNB version of Naive Bayes, which assumes predictors are binary encoded.
from sklearn.naive_bayes import BernoulliNB
from sklearn.cross_validation import cross_val_score, train_test_split

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

/Users/DavisJohnson/anaconda/envs/dsi/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
##### Definitions

def strtimedelta(starttime, stoptime):
    return str(datetime.timedelta(seconds=stoptime-starttime))

def apply_to_all_files(basedir, func=lambda x: x, ext='.h5'):
    cnt = 0
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root, '*'+ext))
        cnt += len(files)
        for f in files:
            func(f)
    return cnt

def func_to_get_artist_name(filename):
    h5 = GETTERS.open_h5_file_read(filename)
    artist_name = GETTERS.get_artist_name(h5)
    all_artist_names.add(artist_name)
    h5.close()

def func_to_count_artist_id(filename):
    h5 = GETTERS.open_h5_file_read(filename)
    artist_id = GETTERS.get_artist_id(h5)
    files_per_artist[artist_id] += 1
    h5.close()   
    

def correlation_heat_map(df):
    corrs = df.corr()

    # Set the default matplotlib figure size:
    fig, ax = plt.subplots(figsize=(12,12))

    # Generate a mask for the upper triangle (taken from seaborn example gallery)
    mask = np.zeros_like(corrs, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Plot the heatmap with seaborn.
    # Assign the matplotlib axis the function returns. This will let us resize the labels.
    ax = sns.heatmap(corrs, mask=mask, annot=True)

    # Resize the labels.
    ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=14, rotation=30)
    ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=14, rotation=0)

    # If you put plt.show() at the bottom, it prevents those useless printouts from matplotlib.
    plt.show()

def update_columns(col_names, label):
    new_names = []
    new_names.append(col_names[0])
    for name in col_names[1:]:
        name = label + name
        new_names.append(name)
    return new_names

def is_ascii(column):
    column_list = []
    for col in column:
        for letter in col:
            if ord(letter) > 128:
                column_list.append(col)
                continue
    return column_list

In [5]:
conn_lyric = sqlite3.connect('../../Capstone/mxm_dataset.db')
conn_addf_metadata = sqlite3.connect('../../Capstone/MillionSongSubset/AdditionalFiles/subset_track_metadata.db')

In [6]:
df_lyric_lyrics = pd.read_sql("SELECT * FROM lyrics", con=conn_lyric)
df_genre = pd.read_csv('../../Capstone/genre.csv')
df_metadata_songs = pd.read_sql("SELECT * FROM songs", con=conn_addf_metadata)

,track_id,mxm_tid,word,count,is_test
0,TRAAAAV128F421A322,4623710,i,6,0
1,TRAAAAV128F421A322,4623710,the,4,0
2,TRAAAAV128F421A322,4623710,you,2,0
3,TRAAAAV128F421A322,4623710,to,2,0
4,TRAAAAV128F421A322,4623710,and,5,0


In [7]:
df_lyric_lyrics_wide = df_lyric_lyrics.pivot_table(index='track_id', 
                                                   columns='word', 
                                                   values='count',
                                                    fill_value=0).reset_index()

In [10]:
df_lyric_lyrics_wide.head()

word,track_id,&,000,1,10,100,12,13,15,16,...,è,é,él,és,était,être,ô,über,–,‘caus
0,TRAAAAV128F421A322,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,TRAAABD128F429CF47,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,TRAAAED128E0783FAB,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,TRAAAEF128F4273421,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,TRAAAEW128F42930C0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df_metadata_songs.head()

,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year
0,TRACCVZ128F4291A8A,Deep Sea Creature,SOVLGJY12A8C13FBED,Call of the Mastodon,ARMQHX71187B9890D3,bc5e2ad6-0a4a-4d90-b911-e9a7e6861727,Mastodon,280.21506,0.780462,0.574275,2001
1,TRACCMH128F428E4CD,No Quieras Marcharte,SOGDQZK12A8C13F37C,Adelante,AR2PT4M1187FB55B1A,d54ea4a6-0b9c-4e47-bed0-289ae9ff4037,Los Chichos,191.68608,0.561590,0.420570,1984
2,TRACCSW128F148C7C3,If I...,SODMVJR12A6D4F985D,Ill Na Na,ARDI88R1187B98DAB2,fd87374e-ffde-4d36-89a8-8a073f795666,Foxy Brown,222.92853,0.687687,0.406686,0
3,TRACCXJ128F428F0CF,Werther - Lyric Drama in four Acts/Act I/Alors...,SOIWBDR12A8C13A4AC,Massenet: Werther,ARUKJUP12086C14589,8a5f2736-bcde-4a2e-8d50-72631d66a7ef,Ramón Vargas;Vladimir Jurowski,278.38649,0.391741,0.291265,0
4,TRACCVS12903D014F8,Ad Te Clamamus Exsvles Mortva Liberi,SOHCCIA12AC907577F,Pentagrammaton,ARZEWUR1187FB53DC8,0be59867-0da4-4e45-9b64-728cdf25487c,Enthroned,89.15546,0.593416,0.395710,2010


In [13]:
df_lyric_lyrics.head()
df_lyric_lyrics.drop('is_test', axis=1, inplace=True)

In [35]:
df_lyric_metadata = pd.merge(df_metadata_songs, df_lyric_lyrics_wide, how='inner', on='track_id')

In [31]:
# ##Cleaning columns to make sure they all follow the same format
# df_lyric_metadata.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)
# ##Removing any letters with columns
# newcols = [''.join([ch for ch in x 
#                     if ch in 'abcdefghijklmnopqrstuvwxyz_0987654321']).decode('ascii','ignore')
#                     for x in df_lyric_metadata.columns]
# df_lyric_metadata.columns = newcols

In [37]:
df_lyric_metadata.to_csv('df_lyric_metadata.csv', encoding='utf-8')

In [48]:
#Convert the genre dataframe into its wide format (based on the track_id)
df_genre['genre_value'] = df_genre['genre_A'].map(lambda x: 1 if type(x) == str else 0)
df_genre_wide = df_genre.pivot_table(index='track_id', 
                                     columns='genre_A', 
                                     values='genre_value',
                                    fill_value=0).reset_index()

In [50]:
df_lyric_genre_metadata = pd.merge(df_lyric_metadata, df_genre_wide, how='inner', on='track_id')

In [54]:
df_lyric_metadata.shape

(2350, 5011)

In [52]:
df_lyric_genre_metadata.shape

(624, 5024)

In [53]:
df_lyric_genre_metadata.to_csv('df_lyric_genre_metadata.csv', encoding='utf-8')

In [ ]:
###Output the genre table into a csv file for EDA

In [56]:
df_genre_wide.head()

genre_A,track_id,Blues,Country,Electronic,Folk,International,Jazz,Latin,New Age,Pop_Rock,Rap,Reggae,RnB,Vocal
0,TRAAAAK128F9318786,0,0,0,0,0,0,0,0,1,0,0,0,0
1,TRAAAAW128F429D538,0,0,0,0,0,0,0,0,0,1,0,0,0
2,TRAAABD128F429CF47,0,0,0,0,0,0,0,0,1,0,0,0,0
3,TRAAAED128E0783FAB,0,0,0,0,0,1,0,0,0,0,0,0,1
4,TRAAAEF128F4273421,0,0,0,0,0,0,0,0,1,0,0,0,0


In [57]:
df_genre_wide.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)

In [58]:
df_genre_wide.to_csv('df_genre_wide.csv', encoding='utf-8')

In [ ]:
#Look into exploration of the lyrics dataframe
df_lyric_lyrics.groupby('word')['count'].sum().sort_values().tail(30).plot(kind="barh")